In [3]:
import tensorflow as tf
import cv2
import numpy as np
import os
from pygame import mixer


In [4]:
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
eye_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_eye.xml')
model = tf.keras.models.load_model('save_models/best_model.h5')

In [8]:
mixer.init()
sound = mixer.Sound('alarm.wav')
cap = cv2.VideoCapture(0)
score = 0

while True:
    _, frame = cap.read()
    height, width = frame.shape[0:2]
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.2, minNeighbors=3)
    eyes = eye_cascade.detectMultiScale(gray, scaleFactor=1.2, minNeighbors=3)

    cv2.rectangle(frame, (0, height-50), (200, height), (0, 0, 0), thickness=cv2.FILLED)

    for(x,y,w,h) in faces:
        cv2.rectangle(frame, (x,y), (x+w, y+h), color=(255, 0, 0), thickness=3)
    
    for((ex,ey,ew,eh)) in eyes:
        # cv2.rectangle(frame, (x,y), (x+w, y+h), color=(0, 255, 0), thickness=3)

        # eye preprocessing steps
        eye = frame[ey:ey+eh, ex:ex+ew] 
        eye = cv2.resize(eye, (80, 80))
        eye = eye/255
        eye = eye.reshape(80, 80, 3)
        eye = np.expand_dims(eye, axis=0)

        # model prediction of the preprocessing eye
        prediction = model.predict(eye)
        print(f'Close Eyes: {np.round(prediction[0][0], 4)}, Open Eyes: {np.round(prediction[0][1], 4)}')

        # If eyes are closed

        if prediction[0][0] > 0.30:
            cv2.putText(frame, 'Closed', (10, height-20), fontFace=cv2.FONT_HERSHEY_COMPLEX_SMALL, fontScale=1, color=(0, 0, 255), thickness=1, lineType=cv2.LINE_AA)
            cv2.putText(frame, 'Score' + str(score), (100, height-20), fontFace=cv2.FONT_HERSHEY_COMPLEX_SMALL, fontScale=1, color=(0, 0, 255), thickness=1, lineType=cv2.LINE_AA)
            score += 1

            if score > 15:
                try:
                    sound.play()
                    # This is a optional line
                    cv2.putText(frame, 'You are Drowsy', (200, height-20), fontFace=cv2.FONT_HERSHEY_COMPLEX_SMALL, fontScale=1, color=(0, 0, 255), thickness=1, lineType=cv2.LINE_AA)
                except:
                    pass
        
        # If eyes are open
        elif prediction[0][1] > 0.90:
            cv2.putText(frame, 'Open', (10, height-20), fontFace=cv2.FONT_HERSHEY_COMPLEX_SMALL, fontScale=1, color=(0, 255, 0), thickness=1, lineType=cv2.LINE_AA)
            cv2.putText(frame, 'Score' + str(score), (100, height-20), fontFace=cv2.FONT_HERSHEY_COMPLEX_SMALL, fontScale=1, color=(0, 255, 0), thickness=1, lineType=cv2.LINE_AA)
            score -= 1
            
            if score < 0:
                score = 0
                
    
    cv2.imshow('Drowsiness Detection', frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

    
    

1/1 [==============================] - 0s 18ms/step
Close Eyes: 0.0, Open Eyes: 1.0
1/1 [==============================] - 0s 17ms/step
Close Eyes: 0.0, Open Eyes: 1.0
1/1 [==============================] - 0s 18ms/step
Close Eyes: 0.015399999916553497, Open Eyes: 0.9846000075340271
1/1 [==============================] - 0s 18ms/step
Close Eyes: 0.2199999988079071, Open Eyes: 0.7799999713897705
1/1 [==============================] - 0s 20ms/step
Close Eyes: 0.2199999988079071, Open Eyes: 0.7799999713897705
1/1 [==============================] - 0s 19ms/step
Close Eyes: 0.00039999998989515007, Open Eyes: 0.9995999932289124
1/1 [==============================] - 0s 18ms/step
Close Eyes: 0.00039999998989515007, Open Eyes: 0.9995999932289124
1/1 [==============================] - 0s 19ms/step
Close Eyes: 0.00139999995008111, Open Eyes: 0.9986000061035156
1/1 [==============================] - 0s 16ms/step
Close Eyes: 0.2655999958515167, Open Eyes: 0.7343999743461609
1/1 [==================

In [10]:
cap.release()
cv2.destroyAllWindows()